\section{Kitaev Hamiltonian}

\begin{equation}
    H_K = \sum_{<i,j>_\alpha} J_\alpha S_i^\alpha S_j^\alpha
\end{equation}
where $\alpha = x, y, z$.

\subsection{Energy spectrum of Kitaev Hamiltonian}

The Ground state of Kitaev model can be exactly solved, with the energy spectrum:
\begin{align}
    E(\vec{k}) &= \pm |f(\vec{k})| \\
    f(\vec{k}) &= 2 \times (J_x e^{i k_1} + J_y e^{i k_2} + J_z )
\end{align}
where $k_1$ and $k_2$ are the components of $\vec{k}$ along $\vec{n}_1$ and $\vec{n}_2$ directions.

\subsection{Ground state energy}

For a $N_1 \times N_2 \times 2$ lattice, the momentums take discrete values:
\begin{equation}
    k_1 = \frac{2 \pi n_1}{N_1}, k_2 = \frac{2 \pi n_2}{N_2}
\end{equation}
where $n_i = 0, ... , n_i -1, i = 1,2 $.

The ground state energy is the summation of all negative energies:
\begin{equation}
    E_G = \sum_{ E(\vec{k}) < 0}  E(\vec{k})
\end{equation}

For isotropic case $ |J_x| = |J_y| = |J_z| = 1$, the following code gives the ground state energy.

In [261]:
import numpy as np
import math

N1 = 2
N2 = 3

Eg = 0
for n1 in range(N1):
    k1 = 2 * math.pi * n1 / N1
    for n2 in range(N2):
        k2 = 2 * math.pi * n2 / N2
        Eg -= 2 * abs(1 + np.exp(1j*k1) + np.exp(1j*k2))

print(Eg)

-18.928203230275507


\section{Basis generation}

We use the eigen states of course we $S^z$ as basis states and use $|0 \rangle$ to represent $|\downarrow \rangle$ and $|1 \rangle$ to represent $|\uparrow \rangle$. The states can thus be represented by a set of binary numbers. Moreover, we can use the corresponding dicimal number of each state as its tag.

For a system with N sites, the dimension of Hilbert space is $2^N$.

\section{Hamiltonian matrix}

we use the 24-sites cluster that preserves the symmetry of honeycomb lattice. 

The Hamiltonian can be rewriten to $H_K = H_x + H_y +H_z$, where $H_z$ is the diagonal term and $H_x, H_y$ are off-diagonal terms:
\begin{align}
      H_x &= \sum_{<i,j>_x} J_x S_i^x S_j^x = \sum_{<i,j>_x} \frac{J_x}{4} ( S_i^+ S_j^- + S_i^- S_j^+ + S_i^+ S_j^+            + S_i^- S_j^-)\\
      H_y &= \sum_{<i,j>_y} J_y S_i^y S_j^y = \sum_{<i,j>_y} \frac{J_y}{4} ( S_i^+ S_j^- + S_i^- S_j^+ - S_i^+ S_j^+            - S_i^- S_j^-)\\
      H_z &= \sum_{<i,j>_z} J_z S_i^z S_j^z
\end{align}
where $S_i^\pm = S_i^x \pm i S_i^y$.

\subsection{$ N_1 \times N_2 \times 2$ lattice}

The lattice vectors are $\vec{n}_1 = (0,1), \vec{n}_2 = (\frac{1}{2},\frac{\sqrt{3}}{2})$. The number of unit cells along these two directions are $N_1$ and $N_2$ respectively. We label the A sublattice atom in the $n$-th unit cell by $2n$ the other by $2n + 1$.

In [54]:
import numpy as np
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import eigsh

def StateConfig(tag):
    '''The configuration of the state with tag.
    
       Setups required: 
           dim: Hilbert space dimension
           SiteNum: total site number
           
       Return: binary form of tag. type: list'''
    if tag >= dim:
        return print('Error: The tag is out of range.')
    else:
        b = bin(tag)[2:]
        b = b.rjust(SiteNum,'0') 
        return b

def flip(tag,i,j):
    '''Flip the spin on i,j site.
    
       Setups required: 
           SiteNum: total site number
           
       Return: The tag of new state, type: int '''
    f = pow(2, SiteNum - i -1) + pow(2, SiteNum - j -1)
    Newtag = tag^f
    return Newtag

def KitaevRhom(Jx, Jy, Jz, periodic = False):
    '''Kitaev Hamiltonian on N1 * N2 * 2 lattice
    
       Setups required: 
           N1, N2: width and length of the rhomboid cluster
           SiteNum: total site number
           dim: Hilbert space dimension
        
        Return: The Non-zero Hamiltonian element H[i] is in the 
                position (row[i], col[i])
                H, row, col : type: list
        '''
    numNZero = 0 # count the number of non-zero element
    H = []
    row = []
    col = []
    for tag in range(dim):
        si = StateConfig(tag) # initial state configuration
        #print('initial configuration=',si,', tag = ',tag)
        
        temp = {} # A temporay dict
        for i in range(N1*N2):
            '''Sx Sx'''
            ntag = flip(tag, 2*i, 2*i+1)
            #sii = StateConfig(ntag)
            #print('Sx Sx |si> =', sii, ', ntag = ',ntag)
            
            if ntag in temp:
                temp[ntag] += Jx /4
            else:
                temp[ntag] = Jx /4
            #print(temp.items())
            
            '''Sy Sy'''
            if i % N1 != 0:
                next = 2*i -1
            elif i % N1 == 0 and periodic == True:
                next = 2 * (i + N1) -1
            else:
                next = 1/3
            if next % 1 == 0:
                ntag = flip(tag, 2*i, next)
                #sii = StateConfig(ntag)
                #print('Sy Sy |si> =', sii, ', ntag = ',ntag)
                if ntag in temp:
                    temp[ntag] -= Jy * (int(si[2*i]) - 0.5) * (int(si[next]) - 0.5)
                else:
                    temp[ntag] = -Jy * (int(si[2*i]) - 0.5) * (int(si[next]) - 0.5)
            
            '''Sz Sz'''
            if i >= N1:
                next = 2*(i - N1) +1
            elif i < N1 and periodic == True:
                next = 2 * (i + N1 * N2 - N1) +1
            else:
                next = 0.5
            if next % 1 == 0:
                if tag in temp:
                    temp[tag] += Jz * (int(si[2*i]) - 0.5) * (int(si[next]) - 0.5)
                else:
                    temp[tag] = Jz * (int(si[2*i]) - 0.5)* (int(si[next]) - 0.5)
            #print(temp.items())
        odTemp = sorted(temp)
        #print('odTemp =', odTemp )
        for sf in odTemp:
            row.append (tag)
            col.append (sf)
            H.append (temp[sf])
            numNZero += 1
      
            
    print ('non zero elemens number = ', numNZero)
    print('Percentage = ', numNZero/(dim * dim) * 100,'%')
    return H, row, col


N1 = 2
N2 = 3
SiteNum = N1 * N2 * 2
dim = pow(2, SiteNum)

Jx = 1
Jy = 1
Jz = 1

H = []
row = []
col = []

H, row, col = KitaevRhom(Jx, Jy, Jz, periodic = True)
#for l in range(len(H)):
    #print('(',row[l],',',col[l],') = ', H[l])

ham = csc_matrix ((H, (row, col)), shape=(dim, dim))
vals, vecs = eigsh (ham, k=5, which = 'SA')

print (vals)

non zero elemens number =  53248
Percentage =  0.3173828125 %
[-2.45007068 -2.45007068 -2.45007068 -2.45007068 -2.34277886]


The other way to generate the Hamiltonain matrix is to write it as the direct product of 